In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import time
import importlib

In [2]:
import sys
sys.path.append("../../../libs_bell")
from lib_preprocessing import job_ids_to_result, flatten_results_jobs_list, arrange_results_list_tensored3, results_list_to_hist_list

In [3]:
with open("pkls/raw_hist_list.pkl", "rb") as f:
    raw_hist_list = pickle.load(f)
with open("pkls/results_meas_cal.pkl", "rb") as f:
    results_meas_cal = pickle.load(f)

In [4]:
import mthree
from qiskit.test.mock import FakeManhattan
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter
from libs_qrem import sgs_algorithm_py

In [5]:
shots = 8192
max_size = 65
max_length = 64
mthree_lu_mitigator_list = []
for n in range(2, max_size + 1):
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 1], mit_pattern=[[i] for i in range(n)])
    mit = mthree.M3Mitigation(FakeManhattan())
    mit.single_qubit_cals = meas_fitter.cal_matrices
    mthree_lu_mitigator_list.append(mit)
    if n % 10 == 0:
        print("size", n, "finished")
print("length of mthree_lu_mitigator_list: ", len(mthree_lu_mitigator_list))

size 10 finished
size 20 finished
size 30 finished
size 40 finished
size 50 finished
size 60 finished
length of mthree_lu_mitigator_list:  64


In [6]:
mthree_lu_hist_list = []
for i in range(max_length):
    meas_layout = [j for j in range(i + 2)]
    t1 = time.perf_counter()
    mthree_lu_hist_list.append( mthree_lu_mitigator_list[i].apply_correction(raw_hist_list[i + 1], 
                                                                       meas_layout, 
                                                                       distance=None,
                                                                       method='direct',
                                                                       max_iter=25, 
                                                                       tol=1e-5,
                                                                       return_mitigation_overhead=True,
                                                                       details=True) )
    t2 = time.perf_counter()
    print(i + 1, "th finished (", i + 2, "qubits )", "total time:", t2 - t1, "s")

1 th finished ( 2 qubits ) total time: 0.0011116659999999001 s
2 th finished ( 3 qubits ) total time: 0.0005872039999985645 s
3 th finished ( 4 qubits ) total time: 0.0006511739999979227 s
4 th finished ( 5 qubits ) total time: 0.005980110999999511 s
5 th finished ( 6 qubits ) total time: 0.0032234410000029357 s
6 th finished ( 7 qubits ) total time: 0.003643530999994482 s
7 th finished ( 8 qubits ) total time: 0.0038080500000035045 s
8 th finished ( 9 qubits ) total time: 0.004294081999987043 s
9 th finished ( 10 qubits ) total time: 0.00422161499999163 s
10 th finished ( 11 qubits ) total time: 0.006154934999997863 s
11 th finished ( 12 qubits ) total time: 0.0061173109999970166 s
12 th finished ( 13 qubits ) total time: 0.008508497999997644 s
13 th finished ( 14 qubits ) total time: 0.012846588000002157 s
14 th finished ( 15 qubits ) total time: 0.01844370099999537 s
15 th finished ( 16 qubits ) total time: 0.021836223000008204 s
16 th finished ( 17 qubits ) total time: 0.0385413540

In [7]:
def expval(hist):
    expval = 0
    shots = 0
    for key, count in hist.items():
        shots += count
        sigma_z = 1
        for s in key:
            if s == "1":
                sigma_z *= -1
        expval += sigma_z * count
    expval /= shots
    return expval

In [8]:
mthree_lu_mitigator_info = []
for i in range(max_length):
    # _ = mthree_lu_mitigator_list[i].apply(raw_hist_list[i + 1])
    prob_dist = sgs_algorithm_py(mthree_lu_hist_list[i][0])
    mthree_lu_mitigator_info.append({# "exact_one_norm_of_reduced_inv_A": mthree_lu_mitigator_list[i].exact_one_norm_of_reduced_inv_A(),
                                 "col_norms": mthree_lu_hist_list[i][1]["col_norms"],
                                 "mitigated_hist": {key: value * shots for key, value in prob_dist.items()},
                                 "x_s": mthree_lu_hist_list[i][0],
                                 # "x_hat": mthree_lu_mitigator_list[i].x_hat(),
                                 "x_tilde": prob_dist,
                                 "sum_of_x": sum(list(mthree_lu_hist_list[i][0].values())),
                                 # "sum_of_x_hat": mthree_lu_mitigator_list[i].sum_of_x_hat(),
                                 "sum_of_x_tilde": sum(list(prob_dist.values())),
                                 # "indices_to_keys_vector": mthree_lu_mitigator_list[i].indices_to_keys_vector(),
                                 "times": {"total": mthree_lu_hist_list[i][1]["time"]},
                                 "expval": expval(prob_dist),
                                 "mitigation_stddev": mthree_lu_hist_list[i][0].stddev(),
                                 })
    print(i + 1, "th finished (", i + 2, "qubits )")

1 th finished ( 2 qubits )
2 th finished ( 3 qubits )
3 th finished ( 4 qubits )
4 th finished ( 5 qubits )
5 th finished ( 6 qubits )
6 th finished ( 7 qubits )
7 th finished ( 8 qubits )
8 th finished ( 9 qubits )
9 th finished ( 10 qubits )
10 th finished ( 11 qubits )
11 th finished ( 12 qubits )
12 th finished ( 13 qubits )
13 th finished ( 14 qubits )
14 th finished ( 15 qubits )
15 th finished ( 16 qubits )
16 th finished ( 17 qubits )
17 th finished ( 18 qubits )
18 th finished ( 19 qubits )
19 th finished ( 20 qubits )
20 th finished ( 21 qubits )
21 th finished ( 22 qubits )
22 th finished ( 23 qubits )
23 th finished ( 24 qubits )
24 th finished ( 25 qubits )
25 th finished ( 26 qubits )
26 th finished ( 27 qubits )
27 th finished ( 28 qubits )
28 th finished ( 29 qubits )
29 th finished ( 30 qubits )
30 th finished ( 31 qubits )
31 th finished ( 32 qubits )
32 th finished ( 33 qubits )
33 th finished ( 34 qubits )
34 th finished ( 35 qubits )
35 th finished ( 36 qubits )
36

In [9]:
with open("./pkls/mthree_lu_mitigator_info.pkl", "wb") as f:
    pickle.dump(mthree_lu_mitigator_info, f)